In [1]:
import pandas as pd
import csv
import numpy
import seaborn as sns
import matplotlib.pyplot as plt

### Read "title.ratings.tsv" file

In [2]:
title_ratings=pd.read_csv("title.ratings.tsv", sep='\t')

In [3]:
title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1647
1,tt0000002,6.1,198
2,tt0000003,6.5,1345
3,tt0000004,6.2,120
4,tt0000005,6.2,2131


### Read "title.basics.tsv" file

In [4]:
title_basics=pd.read_csv("title.basics.tsv", sep='\t')
title_basics=title_basics.drop_duplicates()

/Users/victortulabot/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
title_basics=title_basics[['titleType','tconst','primaryTitle', 'originalTitle', 'startYear']]
title_basics=title_basics[title_basics.titleType=='movie']
title_basics=title_basics[title_basics.startYear.apply(lambda x: str(x).isnumeric())]
title_basics.head()

,titleType,tconst,primaryTitle,originalTitle,startYear
8,movie,tt0000009,Miss Jerry,Miss Jerry,1894
144,movie,tt0000147,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,1897
331,movie,tt0000335,Soldiers of the Cross,Soldiers of the Cross,1900
498,movie,tt0000502,Bohemios,Bohemios,1905
570,movie,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906


### Merge "title.ratings.tsv" and "title.basics.tsv" 

In [6]:
ratings_and_titles=pd.merge(title_ratings.set_index('tconst'), title_basics.set_index('tconst'), left_index=True, right_index=True, how='inner')
ratings_and_titles=ratings_and_titles.drop_duplicates()

In [7]:
ratings_and_titles.head()

,averageRating,numVotes,titleType,primaryTitle,originalTitle,startYear
tconst,,,,,,
tt0000009,5.9,155,movie,Miss Jerry,Miss Jerry,1894
tt0000147,5.2,357,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,1897
tt0000335,6.1,41,movie,Soldiers of the Cross,Soldiers of the Cross,1900
tt0000502,3.8,6,movie,Bohemios,Bohemios,1905
tt0000574,6.1,590,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,1906


### Read netflix_titles.csv

In [8]:
netflix_titles=pd.read_csv("netflix_titles.csv", index_col="show_id")

#### Drop rows without release_year

In [9]:
netflix_titles=netflix_titles.dropna(subset=['release_year'])

#### Change release_year column to integer

In [10]:
netflix_titles.release_year=netflix_titles.release_year.astype(numpy.int64)

#### Drop rows in ratings_and_titles with non-numeric values for startYear and convert to integer

In [11]:
ratings_and_titles=ratings_and_titles[ratings_and_titles.startYear.apply(lambda x: str(x).isnumeric())]

In [12]:
ratings_and_titles.startYear=ratings_and_titles.startYear.astype(numpy.int64)

#### Convert titles to lowercase

In [13]:
netflix_titles['title']=netflix_titles['title'].str.lower()
ratings_and_titles['originalTitle']=ratings_and_titles['originalTitle'].str.lower()
ratings_and_titles['primaryTitle']=ratings_and_titles['primaryTitle'].str.lower()

### Join netflix titles with IMDb ratings on title name and release year.

In [14]:
##subset movies
netflix_titles=netflix_titles[netflix_titles.type=='Movie']

In [15]:
netflix_titles_rating=pd.merge(netflix_titles, ratings_and_titles, left_on=['title', 'release_year'], right_on=['primaryTitle', 'startYear'], how='inner')

### Sort the obtained data frame by averageRating and number of votes

In [16]:
netflix_titles_rating.sort_values(by=['averageRating', 'numVotes'], inplace=True, ascending=False)

In [17]:
netflix_titles_rating_2000=netflix_titles_rating[netflix_titles_rating.numVotes>2000]

In [18]:
netflix_titles_rating_2000.head(30)

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,averageRating,numVotes,titleType,primaryTitle,originalTitle,startYear
1894,Movie,pulp fiction,Quentin Tarantino,"John Travolta, Samuel L. Jackson, Uma Thurman,...",United States,"January 1, 2019",1994,R,154 min,"Classic Movies, Cult Movies, Dramas",This stylized crime caper weaves together stor...,8.9,1782352,movie,pulp fiction,pulp fiction,1994
1854,Movie,the lord of the rings: the return of the king,Peter Jackson,"Elijah Wood, Ian McKellen, Liv Tyler, Viggo Mo...","New Zealand, United States","January 1, 2020",2003,PG-13,201 min,"Action & Adventure, Sci-Fi & Fantasy",Aragorn is revealed as the heir to the ancient...,8.9,1605940,movie,the lord of the rings: the return of the king,the lord of the rings: the return of the king,2003
2836,Movie,schindler's list,Steven Spielberg,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",United States,"April 1, 2018",1993,R,195 min,"Classic Movies, Dramas",Oskar Schindler becomes an unlikely humanitari...,8.9,1184746,movie,schindler's list,schindler's list,1993
1813,Movie,inception,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen...","United States, United Kingdom","January 1, 2020",2010,PG-13,148 min,"Action & Adventure, Sci-Fi & Fantasy, Thrillers","In this mind-bending sci-fi thriller, a man ru...",8.8,2006939,movie,inception,inception,2010
740,Movie,the matrix,"Lilly Wachowski, Lana Wachowski","Keanu Reeves, Laurence Fishburne, Carrie-Anne ...",United States,"November 1, 2019",1999,R,136 min,"Action & Adventure, Sci-Fi & Fantasy",A computer hacker learns that what most people...,8.7,1634375,movie,the matrix,the matrix,1999
1855,Movie,the lord of the rings: the two towers,Peter Jackson,"Elijah Wood, Ian McKellen, Liv Tyler, Viggo Mo...","New Zealand, United States","January 1, 2020",2002,PG-13,179 min,"Action & Adventure, Sci-Fi & Fantasy",Frodo and Sam head to Mordor to destroy the On...,8.7,1451316,movie,the lord of the rings: the two towers,the lord of the rings: the two towers,2002
1971,Movie,be here now,Lilibet Foster,Andy Whitfield,"United States, Australia","February 28, 2017",2015,TV-MA,110 min,Documentaries,Trace the brave journey of actor Andy Whitfiel...,8.7,3157,movie,be here now,be here now,2015
1792,Movie,city of god,"Fernando Meirelles, Katia Lund","Alexandre Rodrigues, Leandro Firmino, Phellipe...","Brazil, France, Germany","January 1, 2020",2002,R,130 min,"Dramas, Independent Movies, International Movies","Growing up in a Rio de Janeiro slum, Rocket is...",8.6,686449,movie,city of god,cidade de deus,2002
2180,Movie,gol maal,Hrishikesh Mukherjee,"Amol Palekar, Bindiya Goswami, Deven Verma, Ut...",India,"December 31, 2019",1979,TV-PG,137 min,"Classic Movies, Comedies, International Movies",When circumstances at work compel Ram to prete...,8.6,17423,movie,gol maal,gol maal,1979
2398,Movie,the departed,Martin Scorsese,"Leonardo DiCaprio, Matt Damon, Jack Nicholson,...",United States,"December 1, 2019",2006,R,151 min,"Dramas, Thrillers",Two rookie Boston cops are sent deep undercove...,8.5,1161114,movie,the departed,the departed,2006


### Split Genres 

In [21]:
from itertools import chain

# return list from series of comma-separated strings
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = netflix_titles_rating_2000.head(100)['listed_in'].str.split(',').map(len)

# create new dataframe, repeating or chaining as appropriate
res = pd.DataFrame({'title': numpy.repeat(netflix_titles_rating_2000.head(100)['title'], lens),
                    'listed_in': chainer(netflix_titles_rating_2000.head(100)['listed_in']),
                    })
res['listed_in']=res['listed_in'].str.strip()

print(res)

                                              title             listed_in
1894                                   pulp fiction        Classic Movies
1894                                   pulp fiction           Cult Movies
1894                                   pulp fiction                Dramas
1854  the lord of the rings: the return of the king    Action & Adventure
1854  the lord of the rings: the return of the king      Sci-Fi & Fantasy
...                                             ...                   ...
1210                                      laal rang  International Movies
1210                                      laal rang             Thrillers
2166                                       bawarchi        Classic Movies
2166                                       bawarchi              Comedies
2166                                       bawarchi                Dramas

[235 rows x 2 columns]


#### Show Top Genres

In [22]:
top_genres=res['listed_in'].value_counts()
top_genres

Dramas                      59
International Movies        47
Action & Adventure          20
Comedies                    19
Documentaries               19
Classic Movies              16
Thrillers                   12
Independent Movies          10
Sci-Fi & Fantasy             8
Romantic Movies              7
Cult Movies                  5
Sports Movies                5
Music & Musicals             3
Children & Family Movies     2
Horror Movies                1
Stand-Up Comedy              1
LGBTQ Movies                 1
Name: listed_in, dtype: int64

### Check for NaN values

In [23]:
netflix_titles_rating_2000.isnull().any()

type             False
title            False
director          True
cast              True
country           True
date_added       False
release_year     False
rating           False
duration         False
listed_in        False
description      False
averageRating    False
numVotes         False
titleType        False
primaryTitle     False
originalTitle    False
startYear        False
year             False
dtype: bool

In [24]:
nan_vars = netflix_titles_rating_2000.columns[netflix_titles_rating_2000.isnull().any()].tolist()
print(nan_vars)

['director', 'cast', 'country']


In [25]:
for variable in nan_vars:
    print(variable, sum(netflix_titles_rating_2000[variable].isnull()))

director 8
cast 53
country 8


In [26]:
netflix_titles_rating_2000['director'].dropna()
netflix_titles_rating_2000['cast'].dropna()
netflix_titles_rating_2000['country'].dropna()

1894                    United States
1854       New Zealand, United States
2836                    United States
1813    United States, United Kingdom
740                     United States
                    ...              
1478            Russia, United States
2525                            India
905                             India
765                             India
1915                    United States
Name: country, Length: 1532, dtype: object

### Compare means of average ratings per rating

In [27]:
netflix_titles_rating_2000.groupby("rating").agg({"averageRating": ["mean", "std"]}).sort_values(("averageRating", "mean"), ascending=False)

averageRating          
                mean       std
rating                        
TV-G        7.500000  0.458258
NR          6.572727  0.933570
R           6.548941  1.004732
G           6.527778  1.388527
TV-PG       6.515190  1.304636
PG-13       6.440000  1.048960
TV-14       6.425352  1.177628
UR          6.280000  0.944458
TV-MA       6.270053  1.022462
PG          6.228070  1.052726
TV-Y7       5.800000  0.435890

### Group release_year into an interval of 10 years

In [30]:
bins = [1941,1951,1961,1971,1981,1991,2001,2011,2020]
labels = ['1941-1950','1951-1960','1961-1970','1971-1980','1981-1990','1991-2000','2001-2010','2011-2020']
netflix_titles_rating_2000['year'] = pd.cut(netflix_titles_rating_2000['release_year'], bins=bins, labels=labels)

netflix_titles_rating_2000.groupby("year").agg({"averageRating": ["mean","std"]}).sort_values(("averageRating", "mean"), ascending=False)

<ipython-input-30-b266762c3ec4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_titles_rating_2000['year'] = pd.cut(netflix_titles_rating_2000['release_year'], bins=bins, labels=labels)


averageRating          
                   mean       std
year                             
1961-1970      7.640000  0.455095
1941-1950      7.400000       NaN
1971-1980      7.387500  0.808817
1951-1960      7.333333  0.631401
1981-1990      7.000000  0.887849
1991-2000      6.693233  0.917275
2001-2010      6.579558  1.054300
2011-2020      6.265606  1.065124

### Compare means of averageRating p